In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import pairwise_distances
from heapq import nsmallest
tf.__version__

'1.15.0'

In [2]:
from bert_serving.client import BertClient
bc = BertClient()

In [3]:
read = pd.read_csv('dataset/test_dataset', delimiter=',', header=None)
data_articles = read.iloc[:,:].values

In [4]:
read = pd.read_csv('dataset/cleanDataTables', delimiter=',', header=None)
data_tables = read.iloc[:,:].values

In [5]:
articles_title = []
articles_id = []

for article_id, title, text in tqdm(data_articles):
    
    articles_id.append(article_id)
    
    articles_title.append(title)

100%|██████████| 10/10 [00:00<00:00, 23224.27it/s]


In [6]:
embedding_articles = bc.encode(articles_title)

In [7]:
article_dense_vector = []

for current_embedding in embedding_articles:
    
    article_dense_vector.append(current_embedding)

In [9]:
tables_title = []

for current_table in tqdm(data_tables):
    
    tables_title.append(str(current_table[1]))

100%|██████████| 10/10 [00:00<00:00, 25528.33it/s]


In [10]:
embedding_tables = bc.encode(tables_title)

In [11]:
tables_dense_vector = []

for current_embedding in embedding_tables:
    
    tables_dense_vector.append(current_embedding)

In [14]:
def get_accuracy(id_ranked_tables, id_query_goal):

    accuracy = 0

    for id_table in id_ranked_tables:
    
        if id_table == id_query_goal:
    
            accuracy = 1
            
            break;

    return accuracy

In [15]:
def save_accuracy(k,accuracy):
    
    if k == 1:
            
        average_top1.append(accuracy)
        
    if k == 5:
            
        average_top5.append(accuracy)
        
    if k == 10:
            
        average_top10.append(accuracy)
        
    if k == 20:
            
        average_top20.append(accuracy)

In [16]:
average_top1 = []
average_top5 = []
average_top10 = []
average_top20 = []

top_k = [1,5,10,20]

for i in tqdm(range(len(article_dense_vector))):
    
    idQueryGoal = articles_id[i]
    
    distance_vector = pairwise_distances(article_dense_vector[i].reshape(1,768), tables_dense_vector, metric='cosine')
    
    #creating the dataframe
    all_tables_score = []
    
    for j in range(len(tables_ids)):
        
        table_id = tables_ids[j]
        table_score = distance_vector[0][j]
        
        new_row = {"table_id": table_id,"table_score": table_score}
        
        all_tables_score.append(new_row)
        
    df_all_tables_scores = pd.DataFrame(all_tables_score)
    df_tables_sorting = df_all_tables_scores.sort_values('table_score')
    
    #compute the accuracy
    for accuracyK in top_k:
        
        selected_top = df_tables_sorting.head(accuracyK)
        min_score = selected_top['table_score'].max()
        draw_tables_socres = df_tables_sorting[df_tables_sorting['table_score'] <= min_score]
        final_ranked_tables = draw_tables_socres.iloc[:,0:1].values
        
        accuracy_value = get_accuracy(final_ranked_tables,idQueryGoal)
        
        #save the accuracy on the list
        save_accuracy(accuracyK,accuracy_value)

100%|██████████| 10/10 [00:00<00:00, 1535.59it/s]


In [17]:
print(str(round(np.mean(average_top1),4)))
print(str(round(np.mean(average_top5),4)))
print(str(round(np.mean(average_top10),4)))
print(str(round(np.mean(average_top20),4)))

0.0 (±) 0.0
0.0 (±) 0.0
0.0 (±) 0.0
0.0 (±) 0.0
